In [ ]:
include("_.jl")

In [ ]:
me_ge_ = OrderedDict{String,Vector{String}}()

In [ ]:
hg = Gene.read_hgnc()

PR_ = string.(hg[hg[!, "locus_group"].=="protein-coding gene", "symbol"])

length(PR_)

## Ensembl

In [ ]:
or = TableAccess.read(joinpath(PAI, "ensembl", "human_to_mouse.tsv.gz"))

or[!, "Gene name"] = Gene.rename(string.(or[!, "Gene name"]))[1]

;

In [ ]:
hu_mo = Dict{String,String}()

for ro in eachrow(or)

    hu = ro["Gene name"]

    mo = ro["Mouse gene name"]

    if !(ismissing(hu) || ismissing(mo))

        hu_mo[hu] = mo

    end

end

println(length(hu_mo))

me_ge_["Ensembl"] = [pr for pr in PR_ if !haskey(hu_mo, pr)]

## Mouse Genome Informatics

In [ ]:
mg = TableAccess.read(joinpath(PAI, "HOM_MouseHumanSequence.rpt.txt.gz"))

se_ = mg[!, "Common Organism Name"] .== "human"

mg[se_, "Symbol"] .= Gene.rename(string.(mg[se_, "Symbol"]))[1]

;

In [ ]:
hu_mo = Dict{String,String}()

for da in groupby(mg, "DB Class Key")

    sy_ = da[!, "Symbol"]

    se_ = da[!, "Common Organism Name"] .== "human"

    for hu in sy_[se_], mo in sy_[.!se_]

        hu_mo[hu] = mo

    end

end

println(length(hu_mo))

me_ge_["MGI"] = [pr for pr in PR_ if !haskey(hu_mo, pr)]

## 2011 De Novo Origin of Human Protein-Coding Genes

In [ ]:
tr_ = readlines(joinpath(PAI, "de_novo_origin_of_human_protein_coding_genes", "genes.txt"))

me_ge_["2011 paper"] = Gene.rename(vcat(["CLLU1", "c22orf45", "DNAH10OS"], tr_))[1]

## 2019 Genes with human-specific features are primarily involved with brain, immune and metabolic evolution

In [ ]:
an_ge_ir = TableAccess.read_xlsx(
    joinpath(
        PAI,
        "genes_with_human_specific_features_are_primarily_involved_with_brain,_immune_and_metabolic_evolution",
        "12859_2019_2886_MOESM2_ESM.xlsx",
    ),
    "HumanSpecific Genes",
)

me_ge_["2019 paper"] = Gene.rename(string.(an_ge_ir[!, "Gene Name"]))[1]

## Intersect

In [ ]:
for ((se1, ge1_), (se2, ge2_)) in combinations(collect(me_ge_), 2)

    gen_ = intersect(ge1_, ge2_)

    n_in = length(gen_)

    println(se1, " ∩ ", se2, " (", n_in, ")")

    for ge in gen_

        println("- ", ge)
    end

    println()

end

## Write

In [ ]:
DictExtension.write(joinpath(PAO, "method_to_genes.json"), sort(me_ge_))

In [ ]:
ge_ = unique(vcat(values(me_ge_)...))

an_ge_me = DataFrame("Gene" => ge_)

for (me, se_) in me_ge_

    an_ge_me[!, me] = [ge in se_ for ge in ge_]

end

an_ge_me[!, "Sum"] = vec(sum(Matrix(an_ge_me[!, 2:end]); dims = 2))

sort!(an_ge_me, "Sum"; rev = true)

TableAccess.write(joinpath(PAO, "human_gene_by_method.tsv"), an_ge_me)

an_ge_me